In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv('../input/data-science-london-scikit-learn/train.csv',header=None)
test_df=pd.read_csv('../input/data-science-london-scikit-learn/test.csv',header=None)
output=pd.read_csv('../input/data-science-london-scikit-learn/trainLabels.csv',header=None)

In [ ]:
train_df.info()
print('-'*40)
test_df.info()

In [ ]:
train_df.describe()

In [ ]:
label_1=0
label_0=0
for i in range (len(output)):
    if output.loc[i,0]==1:
        label_1=label_1+1
    elif output.loc[i,0]==0:
        label_0=label_0+1
print((label_1/1000)*100)
print((label_0/1000)*100)

In [ ]:
from sklearn.mixture import GaussianMixture
x_all = np.r_[train_df,test_df]
print('x_all shape :',x_all.shape)
lowest_bic = np.infty

bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components,covariance_type=cv_type)
        gmm.fit(x_all)
        bic.append(gmm.aic(x_all))
        
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm
            
            
best_gmm.fit(x_all)
gmm_train = best_gmm.predict_proba(train_df)
gmm_test = best_gmm.predict_proba(test_df)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

#Random Forest Classifier
rfc = RandomForestClassifier(random_state=99)

#USING GRID SEARCH
n_estimators = [10, 50, 100, 200,400]
max_depth = [3, 10, 20, 40]
param_grid = dict(n_estimators=n_estimators,max_depth=max_depth)

grid_search_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv = 10,scoring='accuracy',n_jobs=-1).fit(gmm_train, output.values.ravel())
rfc_best = grid_search_rfc.best_estimator_
print('Random Forest Best Score',grid_search_rfc.best_score_)
print('Random Forest Best Parmas',grid_search_rfc.best_params_)
print('Random Forest Accuracy',cross_val_score(rfc_best,gmm_train, output.values.ravel(), cv=10).mean())

In [ ]:
rfc_best.fit(gmm_train,output.values.ravel())
pred  = rfc_best.predict(gmm_test)
mysubmission = pd.DataFrame(pred)

mysubmission.index += 1

mysubmission.columns = ['Solution']
mysubmission['Id'] = np.arange(1,mysubmission.shape[0]+1)
mysubmission = mysubmission[['Id', 'Solution']]

mysubmission.to_csv('Submission.csv',index=False)